In [1]:
import os # Operating System
import numpy as np
import pandas as pd
import datetime as dt # Datetime
import json # library to handle JSON files

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

!conda install -c conda-forge folium=0.5.0 --yes
import folium #import folium # map rendering library

print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Libraries imported.


In [2]:
df_ppd = pd.read_csv("pp-2019.csv")

In [3]:
df_ppd.head(5)

,{8355F009-BD69-55C5-E053-6B04A8C0D090},130000,2019-01-14 00:00,BH2 5PS,O,N,L,44 - 46,Unnamed: 8,POOLE HILL,Unnamed: 10,BOURNEMOUTH,BOURNEMOUTH.1,BOURNEMOUTH.2,B,A
0,{8355F009-BD6C-55C5-E053-6B04A8C0D090},396000,2019-01-31 00:00,SP8 4HB,O,N,F,DOLPHIN INN,NaN,PEACEMARSH,NaN,GILLINGHAM,NORTH DORSET,DORSET,B,A
1,{8355F009-BD6D-55C5-E053-6B04A8C0D090},197500,2019-01-11 00:00,SP8 4EQ,D,N,F,14,NaN,SYLVAN WAY,NaN,GILLINGHAM,NORTH DORSET,DORSET,B,A
2,{8355F009-BD6E-55C5-E053-6B04A8C0D090},230000,2019-01-28 00:00,BH4 9JY,F,N,L,"WHEATON GRANGE, 16",FLAT 6,BRANKSOME WOOD ROAD,NaN,BOURNEMOUTH,BOURNEMOUTH,BOURNEMOUTH,B,A
3,{8355F009-BD6F-55C5-E053-6B04A8C0D090},215500,2019-01-17 00:00,BH12 1AW,F,N,L,344,FLAT 1,POOLE ROAD,BRANKSOME,POOLE,POOLE,POOLE,B,A
4,{8355F009-BD70-55C5-E053-6B04A8C0D090},242000,2019-02-08 00:00,DT11 7FS,O,N,F,22,NaN,RIFLES WAY,NaN,BLANDFORD FORUM,NORTH DORSET,DORSET,B,A


In [4]:
df_ppd.shape

(932878, 16)

In [5]:
df_ppd.columns = ['TUID', 'Price', 'Date_Transfer', 'Postcode', 'Prop_Type', 'Old_New', 'Duration', 'PAON', 'SAON', 'Street', 'Locality', 'Town_City', 'District', 'County', 'PPD_Cat_Type', 'Record_Status']

In [6]:
df_ppd.head()

,TUID,Price,Date_Transfer,Postcode,Prop_Type,Old_New,Duration,PAON,SAON,Street,Locality,Town_City,District,County,PPD_Cat_Type,Record_Status
0,{8355F009-BD6C-55C5-E053-6B04A8C0D090},396000,2019-01-31 00:00,SP8 4HB,O,N,F,DOLPHIN INN,NaN,PEACEMARSH,NaN,GILLINGHAM,NORTH DORSET,DORSET,B,A
1,{8355F009-BD6D-55C5-E053-6B04A8C0D090},197500,2019-01-11 00:00,SP8 4EQ,D,N,F,14,NaN,SYLVAN WAY,NaN,GILLINGHAM,NORTH DORSET,DORSET,B,A
2,{8355F009-BD6E-55C5-E053-6B04A8C0D090},230000,2019-01-28 00:00,BH4 9JY,F,N,L,"WHEATON GRANGE, 16",FLAT 6,BRANKSOME WOOD ROAD,NaN,BOURNEMOUTH,BOURNEMOUTH,BOURNEMOUTH,B,A
3,{8355F009-BD6F-55C5-E053-6B04A8C0D090},215500,2019-01-17 00:00,BH12 1AW,F,N,L,344,FLAT 1,POOLE ROAD,BRANKSOME,POOLE,POOLE,POOLE,B,A
4,{8355F009-BD70-55C5-E053-6B04A8C0D090},242000,2019-02-08 00:00,DT11 7FS,O,N,F,22,NaN,RIFLES WAY,NaN,BLANDFORD FORUM,NORTH DORSET,DORSET,B,A


In [7]:
df_ppd['Date_Transfer'] = df_ppd['Date_Transfer'].apply(pd.to_datetime)

# Delete all obsolete transactions which were done before 2016
df_ppd.drop(df_ppd[df_ppd.Date_Transfer.dt.year < 2016].index, inplace=True)

# Sort by Date of Sale
df_ppd.sort_values(by=['Date_Transfer'],ascending=[False],inplace=True)

In [8]:
df_ppd_man = df_ppd.query("Town_City == 'MANCHESTER'")

# Make a list of street names in MANCHESTER
streets = df_ppd_man['Street'].unique().tolist()

In [9]:
df_grp_price = df_ppd_man.groupby(['Street'])['Price'].mean().reset_index()

# Give meaningful names to the columns
df_grp_price.columns = ['Street', 'Avg_Price']

In [16]:
df_affordable = df_grp_price.query("(Avg_Price >= 1000000) & (Avg_Price <= 2200000)")

In [17]:
df_affordable

,Street,Avg_Price
589,BOOTH STREET,1.769554e+06
636,BOYLE STREET,1.600000e+06
1160,CLAVERTON ROAD,1.800000e+06
1486,DELHI ROAD,1.100000e+06
1530,DEVONSHIRE ROAD,1.099917e+06
2137,GREENSIDE SHOPPING CENTRE,2.100000e+06
2648,JAMES NASMYTH WAY,1.140000e+06
2708,KENNEDY STREET,1.800000e+06
2838,LANDCROSS ROAD,1.003800e+06
3761,OXFORD ROAD,1.944895e+06


In [18]:
import pandas as pd
import numpy as np
import datetime as DT
import hmac
from geopy.geocoders import Nominatim
from geopy.distance import vincenty
# import k-means from clustering stage
from sklearn.cluster import KMeans

In [19]:
for index, item in df_affordable.iterrows():
    print(f"index: {index}")
    print(f"item: {item}")
    print(f"item.Street only: {item.Street}")

index: 589
item: Street       BOOTH STREET
Avg_Price     1.76955e+06
Name: 589, dtype: object
item.Street only: BOOTH STREET
index: 636
item: Street       BOYLE STREET
Avg_Price         1.6e+06
Name: 636, dtype: object
item.Street only: BOYLE STREET
index: 1160
item: Street       CLAVERTON ROAD
Avg_Price           1.8e+06
Name: 1160, dtype: object
item.Street only: CLAVERTON ROAD
index: 1486
item: Street       DELHI ROAD
Avg_Price       1.1e+06
Name: 1486, dtype: object
item.Street only: DELHI ROAD
index: 1530
item: Street       DEVONSHIRE ROAD
Avg_Price        1.09992e+06
Name: 1530, dtype: object
item.Street only: DEVONSHIRE ROAD
index: 2137
item: Street       GREENSIDE SHOPPING CENTRE
Avg_Price                      2.1e+06
Name: 2137, dtype: object
item.Street only: GREENSIDE SHOPPING CENTRE
index: 2648
item: Street       JAMES NASMYTH WAY
Avg_Price             1.14e+06
Name: 2648, dtype: object
item.Street only: JAMES NASMYTH WAY
index: 2708
item: Street       KENNEDY STREET
Avg_Pr

In [20]:
geolocator = Nominatim()

C:\Users\Parth Shah\anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Using Nominatim with the default "geopy/1.21.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  """Entry point for launching an IPython kernel.


In [21]:
df_affordable['city_coord'] = df_affordable['Street'].apply(geolocator.geocode).apply(lambda x: (x.latitude, x.longitude))

C:\Users\Parth Shah\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [22]:
df_affordable

,Street,Avg_Price,city_coord
589,BOOTH STREET,1.769554e+06,"(-27.6201188, 152.8747693)"
636,BOYLE STREET,1.600000e+06,"(53.5488866, -113.4788285)"
1160,CLAVERTON ROAD,1.800000e+06,"(53.3884601, -2.2974684)"
1486,DELHI ROAD,1.100000e+06,"(-33.7927585, 151.1568218)"
1530,DEVONSHIRE ROAD,1.099917e+06,"(42.3235642, -71.2319723)"
2137,GREENSIDE SHOPPING CENTRE,2.100000e+06,"(51.5630155, 0.1216909)"
2648,JAMES NASMYTH WAY,1.140000e+06,"(53.4851508, -2.3579781)"
2708,KENNEDY STREET,1.800000e+06,"(36.7304067, -121.6507991)"
2838,LANDCROSS ROAD,1.003800e+06,"(53.4433904, -2.2202473)"
3761,OXFORD ROAD,1.944895e+06,"(51.5928664, -0.3309175)"


In [23]:
df_affordable[['Latitude', 'Longitude']] = df_affordable['city_coord'].apply(pd.Series)

C:\Users\Parth Shah\anaconda3\lib\site-packages\pandas\core\frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [24]:
df_affordable

,Street,Avg_Price,city_coord,Latitude,Longitude
589,BOOTH STREET,1.769554e+06,"(-27.6201188, 152.8747693)",-27.620119,152.874769
636,BOYLE STREET,1.600000e+06,"(53.5488866, -113.4788285)",53.548887,-113.478829
1160,CLAVERTON ROAD,1.800000e+06,"(53.3884601, -2.2974684)",53.388460,-2.297468
1486,DELHI ROAD,1.100000e+06,"(-33.7927585, 151.1568218)",-33.792758,151.156822
1530,DEVONSHIRE ROAD,1.099917e+06,"(42.3235642, -71.2319723)",42.323564,-71.231972
2137,GREENSIDE SHOPPING CENTRE,2.100000e+06,"(51.5630155, 0.1216909)",51.563015,0.121691
2648,JAMES NASMYTH WAY,1.140000e+06,"(53.4851508, -2.3579781)",53.485151,-2.357978
2708,KENNEDY STREET,1.800000e+06,"(36.7304067, -121.6507991)",36.730407,-121.650799
2838,LANDCROSS ROAD,1.003800e+06,"(53.4433904, -2.2202473)",53.443390,-2.220247
3761,OXFORD ROAD,1.944895e+06,"(51.5928664, -0.3309175)",51.592866,-0.330917


In [25]:
df = df_affordable.drop(columns=['city_coord'])

In [26]:
df

,Street,Avg_Price,Latitude,Longitude
589,BOOTH STREET,1.769554e+06,-27.620119,152.874769
636,BOYLE STREET,1.600000e+06,53.548887,-113.478829
1160,CLAVERTON ROAD,1.800000e+06,53.388460,-2.297468
1486,DELHI ROAD,1.100000e+06,-33.792758,151.156822
1530,DEVONSHIRE ROAD,1.099917e+06,42.323564,-71.231972
2137,GREENSIDE SHOPPING CENTRE,2.100000e+06,51.563015,0.121691
2648,JAMES NASMYTH WAY,1.140000e+06,53.485151,-2.357978
2708,KENNEDY STREET,1.800000e+06,36.730407,-121.650799
2838,LANDCROSS ROAD,1.003800e+06,53.443390,-2.220247
3761,OXFORD ROAD,1.944895e+06,51.592866,-0.330917


In [29]:
address = 'Manchester, UK'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manchester City are {}, {}.'.format(latitude, longitude))

C:\Users\Parth Shah\anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.21.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of Manchester City are 53.4794892, -2.2451148.


In [31]:
map_manchester = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, price, street in zip(df['Latitude'], df['Longitude'], df['Avg_Price'], df['Street']):
    label = '{}, {}'.format(street, price)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manchester)  
    
map_manchester

In [33]:
CLIENT_ID = 'WXWXIASJBMLHW1IZ352OWSFZXNLO5SWTQYPIDOCAVVWVOLYV' # Foursquare ID
CLIENT_SECRET = '1JNZLNARREOIKYQQ1PJ0T2XWCRGIVVLGFAXYJHSQAA1PM2ZG' # Foursquare Secret
VERSION = '20180323' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: WXWXIASJBMLHW1IZ352OWSFZXNLO5SWTQYPIDOCAVVWVOLYV
CLIENT_SECRET:1JNZLNARREOIKYQQ1PJ0T2XWCRGIVVLGFAXYJHSQAA1PM2ZG


In [34]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Street', 
                  'Street Latitude', 
                  'Street Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [35]:
location_venues = getNearbyVenues(names=df['Street'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

BOOTH STREET
BOYLE STREET
CLAVERTON ROAD
DELHI ROAD
DEVONSHIRE ROAD
GREENSIDE SHOPPING CENTRE
JAMES NASMYTH WAY
KENNEDY STREET
LANDCROSS ROAD
OXFORD ROAD
PARK PLACE
PICCADILLY
QUAY STREET
RAKE LANE
RAYNHAM AVENUE
STAKEHILL INDUSTRIAL ESTATE
STANTON AVENUE
STOCKS STREET
THE PADDOCK
TURNSTONE AVENUE
WOODROW WAY


In [36]:
location_venues

,Street,Street Latitude,Street Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,BOOTH STREET,-27.620119,152.874769,Woolworths,-27.622376,152.875716,Grocery Store
1,BOYLE STREET,53.548887,-113.478829,Boyle Street Plaza,53.550017,-113.476313,Park
2,BOYLE STREET,53.548887,-113.478829,Mirama Dining & Lounge,53.545401,-113.477837,Chinese Restaurant
3,BOYLE STREET,53.548887,-113.478829,United Grocers,53.545278,-113.479325,Grocery Store
4,BOYLE STREET,53.548887,-113.478829,Trang Tien,53.545221,-113.478385,Vietnamese Restaurant
...,...,...,...,...,...,...,...
205,TURNSTONE AVENUE,53.454712,-2.629645,Costa Coffee,53.457859,-2.633398,Coffee Shop
206,TURNSTONE AVENUE,53.454712,-2.629645,The Old Crow,53.456667,-2.634394,Bar
207,TURNSTONE AVENUE,53.454712,-2.629645,Newton Cricket Club,53.457977,-2.627325,Cricket Ground
208,TURNSTONE AVENUE,53.454712,-2.629645,newton-le-willows cricket ground,53.458134,-2.627197,Cricket Ground


In [37]:
location_venues.groupby('Street').count()

,Street Latitude,Street Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Street,,,,,,
BOOTH STREET,1,1,1,1,1,1
BOYLE STREET,5,5,5,5,5,5
CLAVERTON ROAD,4,4,4,4,4,4
DELHI ROAD,4,4,4,4,4,4
DEVONSHIRE ROAD,11,11,11,11,11,11
GREENSIDE SHOPPING CENTRE,2,2,2,2,2,2
JAMES NASMYTH WAY,7,7,7,7,7,7
KENNEDY STREET,3,3,3,3,3,3
LANDCROSS ROAD,27,27,27,27,27,27


In [38]:
print('There are {} uniques categories.'.format(len(location_venues['Venue Category'].unique())))

There are 89 uniques categories.


In [39]:
location_venues.shape

(210, 7)

In [40]:
# one hot encoding
venues_onehot = pd.get_dummies(location_venues[['Venue Category']], prefix="", prefix_sep="")

# add street column back to dataframe
venues_onehot['Street'] = location_venues['Street'] 

# move street column to the first column
fixed_columns = [venues_onehot.columns[-1]] + list(venues_onehot.columns[:-1])

#fixed_columns
venues_onehot = venues_onehot[fixed_columns]

venues_onehot.head()

,Street,Afghan Restaurant,Asian Restaurant,Auto Garage,Bagel Shop,Bakery,Bar,Baseball Field,Bistro,Bookstore,...,Sushi Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Train Station,Video Game Store,Vietnamese Restaurant,Warehouse Store
0,BOOTH STREET,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,BOYLE STREET,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,BOYLE STREET,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,BOYLE STREET,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,BOYLE STREET,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [42]:
manchester_grouped = venues_onehot.groupby('Street').mean().reset_index()
manchester_grouped

,Street,Afghan Restaurant,Asian Restaurant,Auto Garage,Bagel Shop,Bakery,Bar,Baseball Field,Bistro,Bookstore,...,Sushi Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Train Station,Video Game Store,Vietnamese Restaurant,Warehouse Store
0,BOOTH STREET,0.0,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.0,0.00000,0.00,0.000000,0.000000,0.0,0.000000
1,BOYLE STREET,0.0,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.0,0.00000,0.00,0.000000,0.000000,0.2,0.000000
2,CLAVERTON ROAD,0.0,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.0,0.00000,0.00,0.000000,0.000000,0.0,0.000000
3,DELHI ROAD,0.0,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.0,0.00000,0.00,0.000000,0.000000,0.0,0.000000
4,DEVONSHIRE ROAD,0.0,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.090909,0.0,0.00000,0.00,0.000000,0.000000,0.0,0.000000
5,GREENSIDE SHOPPING CENTRE,0.0,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.0,0.00000,0.50,0.000000,0.000000,0.0,0.000000
6,JAMES NASMYTH WAY,0.0,0.000000,0.142857,0.00000,0.00,0.000000,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.0,0.00000,0.00,0.142857,0.000000,0.0,0.000000
7,KENNEDY STREET,0.0,0.000000,0.000000,0.00000,0.00,0.000000,0.333333,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.0,0.00000,0.00,0.000000,0.000000,0.0,0.000000
8,LANDCROSS ROAD,0.0,0.074074,0.000000,0.00000,0.00,0.111111,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.0,0.00000,0.00,0.000000,0.000000,0.0,0.000000
9,OXFORD ROAD,0.1,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.1,0.00000,0.00,0.000000,0.000000,0.0,0.000000


In [43]:
manchester_grouped.shape

(19, 90)

In [44]:
# What are the top 5 venues/facilities nearby profitable real estate investments?#

num_top_venues = 5

for hood in manchester_grouped['Street']:
    print("----"+hood+"----")
    temp = manchester_grouped[manchester_grouped['Street'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----BOOTH STREET----
                   venue  freq
0          Grocery Store   1.0
1      Afghan Restaurant   0.0
2         Nursery School   0.0
3                    Pub   0.0
4  Portuguese Restaurant   0.0


----BOYLE STREET----
                   venue  freq
0                   Café   0.2
1  Vietnamese Restaurant   0.2
2     Chinese Restaurant   0.2
3          Grocery Store   0.2
4                   Park   0.2


----CLAVERTON ROAD----
               venue  freq
0           Bus Stop  0.25
1     Sandwich Place  0.25
2        Coffee Shop  0.25
3  Food & Drink Shop  0.25
4        Record Shop  0.00


----DELHI ROAD----
               venue  freq
0         Steakhouse  0.25
1      National Park  0.25
2             Garden  0.25
3               Café  0.25
4  Afghan Restaurant  0.00


----DEVONSHIRE ROAD----
           venue  freq
0  Deli / Bodega  0.18
1  Metro Station  0.18
2   Tennis Court  0.09
3    Coffee Shop  0.09
4    Pizza Place  0.09


----GREENSIDE SHOPPING CENTRE----
              

In [45]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [46]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Street']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

In [48]:
venues_sorted = pd.DataFrame(columns=columns)
venues_sorted['Street'] = manchester_grouped['Street']

for ind in np.arange(manchester_grouped.shape[0]):
    venues_sorted.iloc[ind, 1:] = return_most_common_venues(manchester_grouped.iloc[ind, :], num_top_venues)

In [49]:
venues_sorted.head()

,Street,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,BOOTH STREET,Grocery Store,Warehouse Store,French Restaurant,Cricket Ground,Deli / Bodega,Department Store,Diner,Farmers Market,Fast Food Restaurant,Fish & Chips Shop
1,BOYLE STREET,Vietnamese Restaurant,Grocery Store,Chinese Restaurant,Park,Café,French Restaurant,Department Store,Diner,Farmers Market,Fast Food Restaurant
2,CLAVERTON ROAD,Coffee Shop,Sandwich Place,Food & Drink Shop,Bus Stop,Warehouse Store,Coworking Space,Deli / Bodega,Department Store,Diner,Farmers Market
3,DELHI ROAD,Steakhouse,Café,Garden,National Park,French Restaurant,Deli / Bodega,Department Store,Diner,Farmers Market,Fast Food Restaurant
4,DEVONSHIRE ROAD,Deli / Bodega,Metro Station,Martial Arts Dojo,Pizza Place,Coffee Shop,Neighborhood,Bus Stop,Ice Cream Shop,Tennis Court,Fish & Chips Shop


In [50]:
venues_sorted.shape

(19, 11)

In [51]:
manchester_grouped.shape

(19, 90)

In [52]:
manchester_grouped=df

In [54]:
# set number of clusters
kclusters = 5

manchester_grouped_clustering = manchester_grouped.drop('Street', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manchester_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:50]

array([1, 4, 1, 2, 2, 3, 2, 1, 2, 3, 2, 1, 0, 2, 2, 0, 0, 0, 2, 4, 3])

In [55]:
manchester_grouped_clustering=df
manchester_grouped_clustering.head()

,Street,Avg_Price,Latitude,Longitude
589,BOOTH STREET,1.769554e+06,-27.620119,152.874769
636,BOYLE STREET,1.600000e+06,53.548887,-113.478829
1160,CLAVERTON ROAD,1.800000e+06,53.388460,-2.297468
1486,DELHI ROAD,1.100000e+06,-33.792758,151.156822
1530,DEVONSHIRE ROAD,1.099917e+06,42.323564,-71.231972


In [56]:
manchester_grouped_clustering.shape

(21, 4)

In [57]:
df.shape

(21, 4)

In [58]:
manchester_grouped_clustering.dtypes

Street        object
Avg_Price    float64
Latitude     float64
Longitude    float64
dtype: object

In [59]:
df.dtypes

Street        object
Avg_Price    float64
Latitude     float64
Longitude    float64
dtype: object

In [60]:
# add clustering labels
manchester_grouped_clustering['Cluster Labels'] = kmeans.labels_

# merge london_grouped with london_data to add latitude/longitude for each neighborhood
manchester_grouped_clustering = manchester_grouped_clustering.join(venues_sorted.set_index('Street'), on='Street')

manchester_grouped_clustering.head(30) # check the last columns!"""

,Street,Avg_Price,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
589,BOOTH STREET,1.769554e+06,-27.620119,152.874769,1,Grocery Store,Warehouse Store,French Restaurant,Cricket Ground,Deli / Bodega,Department Store,Diner,Farmers Market,Fast Food Restaurant,Fish & Chips Shop
636,BOYLE STREET,1.600000e+06,53.548887,-113.478829,4,Vietnamese Restaurant,Grocery Store,Chinese Restaurant,Park,Café,French Restaurant,Department Store,Diner,Farmers Market,Fast Food Restaurant
1160,CLAVERTON ROAD,1.800000e+06,53.388460,-2.297468,1,Coffee Shop,Sandwich Place,Food & Drink Shop,Bus Stop,Warehouse Store,Coworking Space,Deli / Bodega,Department Store,Diner,Farmers Market
1486,DELHI ROAD,1.100000e+06,-33.792758,151.156822,2,Steakhouse,Café,Garden,National Park,French Restaurant,Deli / Bodega,Department Store,Diner,Farmers Market,Fast Food Restaurant
1530,DEVONSHIRE ROAD,1.099917e+06,42.323564,-71.231972,2,Deli / Bodega,Metro Station,Martial Arts Dojo,Pizza Place,Coffee Shop,Neighborhood,Bus Stop,Ice Cream Shop,Tennis Court,Fish & Chips Shop
2137,GREENSIDE SHOPPING CENTRE,2.100000e+06,51.563015,0.121691,3,Thrift / Vintage Store,Fish & Chips Shop,Warehouse Store,Fried Chicken Joint,Cricket Ground,Deli / Bodega,Department Store,Diner,Farmers Market,Fast Food Restaurant
2648,JAMES NASMYTH WAY,1.140000e+06,53.485151,-2.357978,2,Train Station,Auto Garage,Pizza Place,Gym,Grocery Store,Sandwich Place,Fast Food Restaurant,Warehouse Store,Food & Drink Shop,Deli / Bodega
2708,KENNEDY STREET,1.800000e+06,36.730407,-121.650799,1,Gym,Baseball Field,Moving Target,Warehouse Store,Fried Chicken Joint,Deli / Bodega,Department Store,Diner,Farmers Market,Fast Food Restaurant
2838,LANDCROSS ROAD,1.003800e+06,53.443390,-2.220247,2,Pub,Bar,Asian Restaurant,Fast Food Restaurant,Coffee Shop,Sandwich Place,Gym / Fitness Center,Greek Restaurant,Fried Chicken Joint,Fish & Chips Shop
3761,OXFORD ROAD,1.944895e+06,51.592866,-0.330917,3,Indian Restaurant,Afghan Restaurant,Sandwich Place,Gym / Fitness Center,Gym,Park,Platform,Convenience Store,Thai Restaurant,Fried Chicken Joint


In [61]:
# Create Map

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manchester_grouped_clustering['Latitude'], manchester_grouped_clustering['Longitude'], manchester_grouped_clustering['Street'], manchester_grouped_clustering['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [62]:
manchester_grouped_clustering.loc[manchester_grouped_clustering['Cluster Labels'] == 0, manchester_grouped_clustering.columns[[1] + list(range(5, manchester_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4019,1322000.0,Clothing Store,Coffee Shop,Bar,Café,Hotel,Pub,Pizza Place,Nightclub,Construction & Landscaping,Farmers Market
4642,1300000.0,Supermarket,Warehouse Store,Fried Chicken Joint,Cricket Ground,Deli / Bodega,Department Store,Diner,Farmers Market,Fast Food Restaurant,Fish & Chips Shop
4667,1395037.0,Liquor Store,Warehouse Store,Fried Chicken Joint,Deli / Bodega,Department Store,Diner,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Food & Drink Shop
4694,1250000.0,Clothing Store,Grocery Store,Fast Food Restaurant,Coffee Shop,Brewery,Warehouse Store,Shopping Plaza,Gym / Fitness Center,Gym,Middle Eastern Restaurant


In [63]:
manchester_grouped_clustering.loc[manchester_grouped_clustering['Cluster Labels'] == 1, manchester_grouped_clustering.columns[[1] + list(range(5, manchester_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
589,1.769554e+06,Grocery Store,Warehouse Store,French Restaurant,Cricket Ground,Deli / Bodega,Department Store,Diner,Farmers Market,Fast Food Restaurant,Fish & Chips Shop
1160,1.800000e+06,Coffee Shop,Sandwich Place,Food & Drink Shop,Bus Stop,Warehouse Store,Coworking Space,Deli / Bodega,Department Store,Diner,Farmers Market
2708,1.800000e+06,Gym,Baseball Field,Moving Target,Warehouse Store,Fried Chicken Joint,Deli / Bodega,Department Store,Diner,Farmers Market,Fast Food Restaurant
3900,1.791515e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [64]:
manchester_grouped_clustering.loc[manchester_grouped_clustering['Cluster Labels'] == 2, manchester_grouped_clustering.columns[[1] + list(range(5, manchester_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1486,1.100000e+06,Steakhouse,Café,Garden,National Park,French Restaurant,Deli / Bodega,Department Store,Diner,Farmers Market,Fast Food Restaurant
1530,1.099917e+06,Deli / Bodega,Metro Station,Martial Arts Dojo,Pizza Place,Coffee Shop,Neighborhood,Bus Stop,Ice Cream Shop,Tennis Court,Fish & Chips Shop
2648,1.140000e+06,Train Station,Auto Garage,Pizza Place,Gym,Grocery Store,Sandwich Place,Fast Food Restaurant,Warehouse Store,Food & Drink Shop,Deli / Bodega
2838,1.003800e+06,Pub,Bar,Asian Restaurant,Fast Food Restaurant,Coffee Shop,Sandwich Place,Gym / Fitness Center,Greek Restaurant,Fried Chicken Joint,Fish & Chips Shop
3783,1.040833e+06,Lounge,Hotel,Nightclub,French Restaurant,Coffee Shop,Irish Pub,Italian Restaurant,Indian Restaurant,Seafood Restaurant,Fast Food Restaurant


In [66]:
manchester_grouped_clustering.loc[manchester_grouped_clustering['Cluster Labels'] == 3, manchester_grouped_clustering.columns[[1] + list(range(5, manchester_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2137,2100000.0,Thrift / Vintage Store,Fish & Chips Shop,Warehouse Store,Fried Chicken Joint,Cricket Ground,Deli / Bodega,Department Store,Diner,Farmers Market,Fast Food Restaurant
3761,1944895.0,Indian Restaurant,Afghan Restaurant,Sandwich Place,Gym / Fitness Center,Gym,Park,Platform,Convenience Store,Thai Restaurant,Fried Chicken Joint
5386,2000000.0,Grocery Store,Warehouse Store,French Restaurant,Cricket Ground,Deli / Bodega,Department Store,Diner,Farmers Market,Fast Food Restaurant,Fish & Chips Shop


In [68]:
manchester_grouped_clustering.loc[manchester_grouped_clustering['Cluster Labels'] == 4, manchester_grouped_clustering.columns[[1] + list(range(5, manchester_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
636,1600000.0,Vietnamese Restaurant,Grocery Store,Chinese Restaurant,Park,Café,French Restaurant,Department Store,Diner,Farmers Market,Fast Food Restaurant
4965,1475000.0,Cricket Ground,Coffee Shop,Bar,Fried Chicken Joint,Deli / Bodega,Department Store,Diner,Farmers Market,Fast Food Restaurant,Fish & Chips Shop
